In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shl-dataset/dataset/sample_submission.csv
/kaggle/input/shl-dataset/dataset/train.csv
/kaggle/input/shl-dataset/dataset/test.csv
/kaggle/input/shl-dataset/dataset/audios_test/audio_885.wav
/kaggle/input/shl-dataset/dataset/audios_test/audio_698.wav
/kaggle/input/shl-dataset/dataset/audios_test/audio_1176.wav
/kaggle/input/shl-dataset/dataset/audios_test/audio_1215.wav
/kaggle/input/shl-dataset/dataset/audios_test/audio_66.wav
/kaggle/input/shl-dataset/dataset/audios_test/audio_386.wav
/kaggle/input/shl-dataset/dataset/audios_test/audio_1026.wav
/kaggle/input/shl-dataset/dataset/audios_test/audio_330.wav
/kaggle/input/shl-dataset/dataset/audios_test/audio_72.wav
/kaggle/input/shl-dataset/dataset/audios_test/audio_858.wav
/kaggle/input/shl-dataset/dataset/audios_test/audio_107.wav
/kaggle/input/shl-dataset/dataset/audios_test/audio_820.wav
/kaggle/input/shl-dataset/dataset/audios_test/audio_300.wav
/kaggle/input/shl-dataset/dataset/audios_test/audio_435.wav
/kaggle/input/sh

In [1]:
!pip install git+https://github.com/openai/whisper.git
!pip install transformers
!pip install language-tool-python
!apt-get install -y openjdk-11-jre


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-4mdx583e
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-4mdx583e
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 5.6 MB/s eta 0:00:000:00:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803707 sha256=ed41059fe467488065947af89f5ce366710bdaad6885f8a31833f5fd8a656fbd
  Stored in directory: /tmp/pip-ephem-wheel-cache-fwcf15_t/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 2.4 MB

In [3]:
!pip install git+https://github.com/openai/whisper.git
!pip install transformers
!pip install language-tool-python
!apt-get install -y openjdk-11-jre


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-n511ddvw
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-n511ddvw
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.7 MB/s eta 0:00:000:00:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803707 sha256=799c8964db4c7f6aa37b97f3bb371cbc9b8795946ea52d163550a4f6d48d16ee
  Stored in directory: /tmp/pip-ephem-wheel-cache-ehx7pefc/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 1.8 MB

In [4]:
import os
import librosa
import numpy as np
import pandas as pd
import whisper
import language_tool_python
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tqdm import tqdm


In [5]:
AUDIO_DIR = '/kaggle/input/shl-dataset/dataset/audios_train'
train_csv = pd.read_csv('/kaggle/input/shl-dataset/dataset/train.csv')


In [6]:
asr_model = whisper.load_model("small")
tool = language_tool_python.LanguageTool('en-US')


100%|███████████████████████████████████████| 461M/461M [00:07<00:00, 65.5MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [7]:
def extract_audio_features(path):
    y, sr = librosa.load(path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)
    return mfcc_mean

def transcribe_and_score(path):
    result = asr_model.transcribe(path)
    text = result['text']
    matches = tool.check(text)
    return text, len(matches), len(set(match.ruleId for match in matches))


In [8]:
data = []

for fname in tqdm(sorted(os.listdir(AUDIO_DIR))):
    if fname.endswith('.wav'):
        full_path = os.path.join(AUDIO_DIR, fname)
        text, grammar_errors, unique_errors = transcribe_and_score(full_path)
        mfcc_feat = extract_audio_features(full_path)
        data.append({
            'filename': fname,
            'text': text,
            'grammar_errors': grammar_errors,
            'unique_errors': unique_errors,
            'mfcc': mfcc_feat
        })

df = pd.DataFrame(data)


100%|██████████| 444/444 [1:31:27<00:00, 12.36s/it]  


In [9]:
full_df = pd.merge(df, train_csv, on='filename')  # Has column 'grammar_score'
full_df.head()


,filename,text,grammar_errors,unique_errors,mfcc,label
0,audio_1008.wav,The screw playground is lively and filled wit...,1,1,"[-438.01587, 36.447296, 3.8215935, 18.46732, -...",4.0
1,audio_1017.wav,"In the school playground, the midday sun catc...",2,2,"[-320.42764, 123.34535, -19.073486, 23.02029, ...",4.0
2,audio_102.wav,"At the market, there's a lot of people sellin...",6,3,"[-317.47037, 96.71045, 9.292677, 30.722069, -4...",5.0
3,audio_1024.wav,during the 16th year we had to go through Mas...,7,2,"[-395.97574, 83.70689, -9.338621, 22.242302, 2...",1.5
4,audio_1025.wav,I would like to go to the Sachi for the Sachi...,9,1,"[-316.61896, 65.66385, 3.239592, 33.875946, -1...",2.0


In [14]:
train_csv = pd.read_csv('/kaggle/input/shl-dataset/dataset/train.csv')
print(train_csv.columns)


Index(['filename', 'label'], dtype='object')


In [16]:
# Separate features
audio_features = np.stack(full_df['mfcc'].values)
grammar_features = full_df[['grammar_errors', 'unique_errors']].values
X = np.concatenate([audio_features, grammar_features], axis=1)

# Target
y = full_df['label'].values


# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [25]:
from tensorflow.keras.losses import MeanSquaredError
# Model definition
input_layer = Input(shape=(X_train.shape[1],))
x = Dense(128, activation='relu')(input_layer)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
x = Dense(1)(x)

model = Model(inputs=input_layer, outputs=x)
model.compile(optimizer='adam', loss=MeanSquaredError(), metrics=['mae'])

model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val),
          callbacks=[EarlyStopping(patience=3)], batch_size=16)


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 12.1894 - mae: 3.2492 - val_loss: 3.1141 - val_mae: 1.4336
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.9324 - mae: 1.3654 - val_loss: 1.5061 - val_mae: 0.9804
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.9610 - mae: 1.1426 - val_loss: 1.3359 - val_mae: 0.9323
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.7705 - mae: 1.0824 - val_loss: 1.3409 - val_mae: 0.9122
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.7638 - mae: 1.0989 - val_loss: 1.2427 - val_mae: 0.8750
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4002 - mae: 0.9655 - val_loss: 1.2048 - val_mae: 0.8647
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4953 - mae: 0.9711 - val_loss: 1.1991 - val_mae: 0.8616
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3562 - mae: 0.9423 - val_loss: 1.1730 - val_mae: 0.8600
Epoch 9/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3308 - mae

In [26]:
model.save('/kaggle/working/grammar_score_model.h5')
import joblib
joblib.dump(scaler, '/kaggle/working/scaler.pkl')


['/kaggle/working/scaler.pkl']

In [27]:
from tensorflow.keras.models import load_model
import joblib

model = load_model('/kaggle/working/grammar_score_model.h5')
scaler = joblib.load('/kaggle/working/scaler.pkl')


In [28]:
TEST_AUDIO_DIR = '/kaggle/input/shl-dataset/dataset/audios_test'
sample_sub = pd.read_csv('/kaggle/input/shl-dataset/dataset/sample_submission.csv')


In [29]:
test_data = []

for fname in tqdm(sample_sub['filename']):
    full_path = os.path.join(TEST_AUDIO_DIR, fname)
    text, grammar_errors, unique_errors = transcribe_and_score(full_path)
    mfcc_feat = extract_audio_features(full_path)
    test_data.append({
        'filename': fname,
        'grammar_errors': grammar_errors,
        'unique_errors': unique_errors,
        'mfcc': mfcc_feat
    })

test_df = pd.DataFrame(test_data)


100%|██████████| 195/195 [39:43<00:00, 12.22s/it]  


In [30]:
audio_features_test = np.stack(test_df['mfcc'].values)
grammar_features_test = test_df[['grammar_errors', 'unique_errors']].values
X_test = np.concatenate([audio_features_test, grammar_features_test], axis=1)

# Scale
X_test_scaled = scaler.transform(X_test)


In [31]:
preds = model.predict(X_test_scaled).flatten()

sample_sub['grammar_score'] = preds.clip(0, 5)  # Keep within 0-5 range
sample_sub.to_csv('/kaggle/working/sample_submission.csv', index=False)
sample_sub.head()


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


,filename,label,grammar_score
0,audio_706.wav,0,2.911711
1,audio_800.wav,0,2.870077
2,audio_68.wav,0,4.674707
3,audio_1267.wav,0,4.008883
4,audio_683.wav,0,3.364947
